# ==========================================
# 1. ライブラリのインポートと設定
# ==========================================

In [ ]:
# 🔧 環境セットアップ (Google Drive マウント版)
# =================================================
import os
import sys
import torch

# 1. Google Driveをマウント
print("🔄 Google Driveをマウント中...")
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Driveマウント完了")
except Exception as e:
    print(f"❌ Google Driveマウントエラー: {e}")
    raise

# 2. プロジェクトのパスを設定
# ※Google Drive内のプロジェクトフォルダパスを指定してください
PROJECT_PATH = '/content/drive/MyDrive/sae_pj2'

if not os.path.exists(PROJECT_PATH):
    print(f"❌ プロジェクトフォルダが見つかりません: {PROJECT_PATH}")
    print("💡 PROJECT_PATHの変数が正しいか、フォルダが同期されているか確認してください。")
else:
    print(f"✅ プロジェクトフォルダ発見: {PROJECT_PATH}")

    # 3. Pythonがプロジェクト内の.pyファイルを読み込めるようにパスを追加
    if PROJECT_PATH not in sys.path:
        sys.path.insert(0, PROJECT_PATH)
        print(f"   🐍 Pythonパスに追加しました")

    # colabフォルダもパスに追加
    colab_path = os.path.join(PROJECT_PATH, 'colab')
    if colab_path not in sys.path:
        sys.path.insert(0, colab_path)
        print(f"   🐍 colabパスに追加しました")

    # 4. 作業ディレクトリをプロジェクトフォルダに移動
    os.chdir(PROJECT_PATH)
    print(f"   📁 作業ディレクトリを変更しました: {os.getcwd()}")

    # 4.5 必須ファイルの存在チェック
    required_items = [
        ('config.py', '設定ファイル'),
        ('colab/feedback_analyzer.py', 'メインロジック'),
        ('eval_dataset', 'データセットフォルダ'),
        ('eval_dataset/feedback.jsonl', 'Feedbackデータ')
    ]
    for path, label in required_items:
        exists = os.path.exists(path)
        icon = '✅' if exists else '⚠️'
        kind = '📄' if os.path.isfile(path) else '📁'
        print(f"   {icon} 必須確認: {kind} {path} - {label}{'' if exists else '（見つからない場合は同期を確認）'}")

# 5. 依存関係のインストール
print("\n📦 依存関係をインストール中...")
req_file = 'requirements-colab.txt'
if os.path.exists(req_file):
    try:
        get_ipython().system(f"pip install -q -r {req_file}")
        print("✅ 依存関係インストール完了 (requirements-colab.txtを使用)")
    except Exception as e:
        print(f"⚠️ 依存関係インストールで警告/エラー: {e}")
        print("   必要に応じて個別に !pip install <package> を実行してください")
else:
    print(f"⚠️ {req_file} が見つかりません。必要なライブラリが不足する可能性があります。")

# 6. 環境確認
print("\n🚀 環境セットアップ完了！")
if torch.cuda.is_available():
    try:
        print(f"   GPU検出: {torch.cuda.get_device_name(0)}")
        memory_total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"   GPU Memory: {memory_total:.1f}GB")
    except Exception:
        print("   GPU検出: 利用可能 (デバイス名の取得に失敗)")
else:
    print("   ❌ GPU利用不可 (CPUモード)")


In [ ]:
import json
import gc
from typing import Dict, Any, Generator
from pathlib import Path
from tqdm import tqdm
from transformer_lens import HookedTransformer
from sae_lens import SAE

# メモリ最適化設定
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
torch.set_grad_enabled(True) # 勾配計算を有効化

In [ ]:
from huggingface_hub import login
login()  # 画面の指示に従いトークン入力

# ==========================================
# 2. データ処理ロジック (指示書に基づく実装)
# ==========================================

In [ ]:
def yield_sycophancy_samples(data: Dict[str, Any]) -> Generator[Dict[str, Any], None, None]:
    """
    JSONデータからAttribution Patching用のサンプルを生成するジェネレータ
    
    新旧両方のファイル構造に対応:
    - 旧形式: variations直下にprompt/responseがある
    - 新形式: metadataセクションがあり、sae_activationsなど新構造（template_type=""がbase）
    """
    results = data.get("results", [])

    for result in results:
        variations = result.get("variations", [])
        question_id = result.get("question_id")

        # 1. Base回答の特定（空文字列もbaseとみなす）
        base_variation = None
        for var in variations:
            t_type = var.get("template_type")
            if t_type == "base" or t_type == "(base)" or t_type == "" or not t_type:
                base_variation = var
                break

        if not base_variation:
            continue

        # 2. ターゲット（迎合）回答の特定とペアリング
        for idx, target_variation in enumerate(variations):
            if target_variation is base_variation:
                continue

            if target_variation.get("sycophancy_flag") == 1:
                yield {
                    "question_id": question_id,
                    "variation_index": idx,
                    "template_type": target_variation.get("template_type"),
                    "prompt": target_variation.get("prompt"),
                    "target_response": target_variation.get("response"),
                    "base_response": base_variation.get("response")
                }

# ==========================================
# 3. Attribution Patching 分析クラス
# ==========================================

In [ ]:
class AttributionPatchingAnalyzer:
    def __init__(self, model: HookedTransformer, sae: SAE, config: Any):
        self.model = model
        self.sae = sae
        self.config = config
        # hook_name を直接指定（config.py から取得するのが理想）
        # Gemma Scope SAE の sae_id から推定
        # 例: "layer_31/width_16k/canonical" → "blocks.31.hook_resid_post"
        sae_id = config.model.sae_id
        layer_num = sae_id.split('/')[0].replace('layer_', '')
        self.hook_name = f"blocks.{layer_num}.hook_resid_post"
        print(f"   🎯 Using hook: {self.hook_name}")

    def _find_answer_start_position(self, full_tokens: torch.Tensor, prompt_str: str) -> int:
        """
        プロンプトの終わり（回答の始まり）のトークン位置を特定する
        """
        # プロンプト単体でのトークン長を取得
        # Note: BOSトークン等の扱いに注意。Gemmaはadd_bos_token=Trueがデフォルト
        prompt_tokens = self.model.to_tokens(prompt_str, prepend_bos=True)
        return prompt_tokens.shape[1] - 1  # 0-indexed なので -1 (最後のトークン位置)

    def calculate_atp_for_sample(self, sample: Dict[str, Any]) -> Dict[str, Any]:
        """
        1サンプルに対するAttribution Patchingを実行
        """
        prompt = sample["prompt"]
        response = sample["target_response"]
        base_response = sample["base_response"]

        # 1. トークン化 (Teacher Forcing Input)
        # プロンプト + 実際の回答
        full_text = prompt + response
        input_tokens = self.model.to_tokens(full_text, prepend_bos=True)

        # 回答開始位置（プロンプトの最後のトークン位置）を特定
        # ここが「次のトークン（回答の1文字目）」を予測する位置になる
        target_pos = self._find_answer_start_position(input_tokens, prompt)

        # 入力長チェック (モデルのコンテキスト長を超えないか)
        if input_tokens.shape[1] > self.model.cfg.n_ctx:
            return {"error": "Sequence too long"}

        # 2. Target Token と Base Token の ID を取得
        # Base回答をトークン化
        base_full_text = prompt + base_response
        base_input_tokens = self.model.to_tokens(base_full_text, prepend_bos=True)
        base_target_pos = self._find_answer_start_position(base_input_tokens, prompt)

        # 複数トークン位置を試して、異なるトークンペアを見つける
        max_tokens_to_check = 20  # 最大20トークンまでチェック
        target_token_id = None
        base_token_id = None
        token_offset = 1  # プロンプト直後からスタート

        for offset in range(1, max_tokens_to_check + 1):
            try:
                candidate_target = input_tokens[0, target_pos + offset].item()
                candidate_base = base_input_tokens[0, base_target_pos + offset].item()

                # 異なるトークンが見つかったら採用
                if candidate_target != candidate_base:
                    target_token_id = candidate_target
                    base_token_id = candidate_base
                    token_offset = offset
                    break
            except IndexError:
                # どちらかの回答が短すぎる場合
                break

        # すべてのトークンが同一、または取得失敗の場合
        if target_token_id is None or base_token_id is None:
            return {"skipped": "No differing tokens found in first 5 positions"}

        # 使用するトークン位置を更新（Logit取得用）
        # target_pos はプロンプト最後の位置なので、offset-1 の位置のLogitを見る
        logit_pos = target_pos + token_offset - 1

        # 3. Forward Pass & Metric Calculation
        self.model.eval()
        self.model.zero_grad()

        # フック内でデータをキャプチャするためのコンテナ
        feature_acts_storage = {}

        def atp_hook(activation, hook):
            """
            Activationを取得し、SAEを通して勾配を流すフック
            """
            # activation: [batch, seq, d_model]
            # ターゲット位置（回答直前）のみを抽出
            # batch=1 前提
            target_act = activation[:, target_pos:target_pos+1, :]

            # SAE Encode (Feature Activation計算)
            # SAEの入力次元に合わせて調整
            f_acts = self.sae.encode(target_act) # [1, 1, n_features]

            # 勾配計算のために保存 (retain_grad重要)
            f_acts.requires_grad_(True)
            f_acts.retain_grad()
            feature_acts_storage['acts'] = f_acts

            # SAE Decode (Reconstruction)
            x_hat = self.sae.decode(f_acts)

            # Gradient Trick:
            # Forward: 元のActivation (x) をそのまま流す (Teacher Forcingの精度維持)
            # Backward: Reconstruction (x_hat) を通して勾配を流す (SAE特徴量へのPathを作る)
            # x_out = x_hat + (x - x_hat).detach()
            # これにより、Metricの勾配は x_hat -> f_acts と伝播する

            x_out = x_hat + (target_act - x_hat).detach()

            # 元のシーケンスに戻す
            activation[:, target_pos:target_pos+1, :] = x_out
            return activation

        # モデル実行
        try:
            logits = self.model.run_with_hooks(
                input_tokens,
                fwd_hooks=[(self.hook_name, atp_hook)]
            )

            # 4. Metric Calculation (Logit Difference)
            # logit_pos の位置での予測を見る（offsetに応じた位置）
            target_logit = logits[0, logit_pos, target_token_id]
            base_logit = logits[0, logit_pos, base_token_id]
            metric = target_logit - base_logit

            # 5. Backward Pass
            metric.backward()

            # 6. AtP Score Calculation
            # Score = Activation * Gradient
            f_acts = feature_acts_storage['acts']
            f_grad = f_acts.grad

            if f_acts is None or f_grad is None:
                return {"error": "Failed to capture gradients"}

            atp_scores = (f_acts * f_grad).detach().cpu().squeeze() # [n_features]

            # 結果の抽出（Top-K & Non-zero）
            # メモリ節約のため、スコアが高いものだけを保存
            top_k = 50
            top_indices = torch.topk(atp_scores.abs(), k=top_k).indices

            top_features = []
            for idx in top_indices:
                idx_val = idx.item()
                score = atp_scores[idx_val].item()
                activation_val = f_acts[0, 0, idx_val].item()

                top_features.append({
                    "id": str(idx_val),
                    "score": score,
                    "activation": activation_val,
                    "gradient": f_grad[0, 0, idx_val].item()
                })

            return {
                "status": "success",
                "target_token": self.model.to_string(target_token_id),
                "base_token": self.model.to_string(base_token_id),
                "token_position": token_offset,  # どの位置のトークンを使ったか記録
                "logit_diff": metric.item(),
                "top_features": top_features
            }

        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                torch.cuda.empty_cache()
                return {"error": "OOM"}
            raise e
        finally:
            # メモリクリーンアップ
            self.model.zero_grad()
            del feature_acts_storage
            if 'logits' in locals(): del logits
            if 'f_acts' in locals(): del f_acts
            if 'f_grad' in locals(): del f_grad
            torch.cuda.empty_cache()

# ==========================================
# 4. メイン実行スクリプト
# ==========================================

In [ ]:
def run_attribution_patching_pipeline(target_layer: int = 31, input_json_path: str = None):
    """
    Attribution Patching分析のメインパイプライン
    
    Args:
        target_layer: 分析対象の層 (9, 16, 20, 31)
        input_json_path: 入力JSONファイルのパス（指定しない場合は自動探索）
    """
    # パス設定
    if input_json_path is None:
        # labeled_dataフォルダから最新のファイルを探す
        search_dir = Path(os.path.join(PROJECT_PATH, "results/labeled_data"))
        if search_dir.exists():
            json_files = list(search_dir.glob("*.json"))
            if json_files:
                input_json_path = str(sorted(json_files)[-1])
                print(f"📂 自動検出: {input_json_path}")
        
        if input_json_path is None:
            raise FileNotFoundError("入力JSONファイルが見つかりません。input_json_pathを指定してください。")
    
    INPUT_JSON_PATH = input_json_path

    # タイムスタンプ付きの出力ファイル名を生成（層情報を含む）
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    OUTPUT_JSON_PATH = os.path.join(
        PROJECT_PATH,
        f"results/feedback/atp_results_gemma-2-9b-it_layer{target_layer}_{timestamp}.json"
    )
    
    print(f"🎯 Target Layer: {target_layer}")

    # ファイル存在確認
    # 層に応じたConfigを選択
    from config import (
        FEEDBACK_GEMMA2_9B_IT_LAYER9_CONFIG,
        FEEDBACK_GEMMA2_9B_IT_LAYER20_CONFIG,
        FEEDBACK_GEMMA2_9B_IT_CONFIG  # Layer 31
    )
    
    config_map = {
        9: FEEDBACK_GEMMA2_9B_IT_LAYER9_CONFIG,
        16: None,  # 16層の設定が必要な場合は config.py に追加してください
        20: FEEDBACK_GEMMA2_9B_IT_LAYER20_CONFIG,
        31: FEEDBACK_GEMMA2_9B_IT_CONFIG
    }
    
    selected_config = config_map.get(target_layer)
    if selected_config is None:
        # 16層など未定義の場合は手動で設定を作成
        if target_layer == 16:
            print(f"⚠️ Layer {target_layer}の設定がないため、動的に生成します")
            from config import ExperimentConfig, ModelConfig, DataConfig, GenerationConfig
            from config import PromptConfig, AnalysisConfig, VisualizationConfig, DebugConfig, FeedbackConfig
            selected_config = ExperimentConfig(
                model=ModelConfig(
                    name="gemma-2-9b-it",
                    sae_release="gemma-scope-9b-it-res-canonical",
                    sae_id=f"layer_{target_layer}/width_16k/canonical",
                    hook_name=f"blocks.{target_layer}.hook_resid_post",
                    device="cuda",
                    use_accelerate=True,
                    use_fp16=False,
                    use_bfloat16=True,
                    low_cpu_mem_usage=True,
                    device_map="auto"
                )
            )
        else:
            raise ValueError(f"Layer {target_layer}はサポートされていません。9, 16, 20, 31のいずれかを指定してください。")

    print(f"🔄 Loading Model & SAE for Layer {target_layer}...")
    device = "cuda" if torch.cuda.is_available() else "cpu"

    model = HookedTransformer.from_pretrained_no_processing(
        selected_config.model.name,
        device=device,
        dtype=torch.bfloat16
    )

    # SAEロード
    sae = SAE.from_pretrained(
        release=selected_config.model.sae_release,
        sae_id=selected_config.model.sae_id,
        device=device
    )

    analyzer = AttributionPatchingAnalyzer(model, sae, selected_config)

    # 1. データの読み込み
    print(f"📂 Loading data from {INPUT_JSON_PATH}...")
    with open(INPUT_JSON_PATH, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 2. メインループ
    samples = list(yield_sycophancy_samples(data))
    print(f"🚀 Starting ATP analysis for {len(samples)} samples...")
    print(f"💾 Results will be saved to: {OUTPUT_JSON_PATH}")

    for i, sample in enumerate(tqdm(samples)):
        res = analyzer.calculate_atp_for_sample(sample)

        # 元のJSONに結果を統合
        question_id = sample["question_id"]
        variation_idx = sample["variation_index"]

        # デバッグ: 結果の内容を表示（最初の数サンプルのみ）
        if i < 3:
            print(f"\n🔍 Sample {i} result: {res}")

        # 該当するvariationを探して atp_analysis フィールドを追加
        for result in data["results"]:
            if result["question_id"] == question_id:
                variations = result["variations"]
                if variation_idx < len(variations):
                    if res.get("status") == "success":
                        variations[variation_idx]["atp_analysis"] = {
                            "top_features": res["top_features"],
                            "target_token": res["target_token"],
                            "base_token": res["base_token"],
                            "token_position": res["token_position"],
                            "logit_diff": res["logit_diff"]
                        }
                    else:
                        # エラーの場合、詳細情報も保存
                        variations[variation_idx]["atp_analysis"] = {
                            "error": res.get("error") or res.get("skipped") or "unknown",
                            "details": res
                        }
                        # 最初のエラーを表示
                        if i < 10:
                            print(f"⚠️ Sample {i} (Q{question_id}, Var{variation_idx}): {res}")
                break

        # 定期的に保存
        if (i + 1) % 10 == 0:
            with open(OUTPUT_JSON_PATH, 'w', encoding='utf-8') as f:
                json.dump(data, f, indent=2, ensure_ascii=False)
            gc.collect()
            torch.cuda.empty_cache()

    # 最終保存
    with open(OUTPUT_JSON_PATH, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

    print(f"✅ Analysis completed. Saved to {OUTPUT_JSON_PATH}")

In [ ]:
# 実行
if __name__ == "__main__":
    # Colab環境での実行を想定
    # 
    # 使用例:
    # run_attribution_patching_pipeline(target_layer=31)  # 31層で実行
    # run_attribution_patching_pipeline(target_layer=20, input_json_path="/path/to/file.json")  # 20層、ファイル指定
    
    # 📌 ここで層を選択してください (9, 16, 20, 31)
    TARGET_LAYER = 31
    
    # 📌 入力ファイルを指定する場合はここを変更（Noneの場合は自動検出）
    INPUT_FILE = None  # 例: "/content/drive/MyDrive/sae_pj2/results/labeled_data/xxx.json"
    
    try:
        run_attribution_patching_pipeline(
            target_layer=TARGET_LAYER,
            input_json_path=INPUT_FILE
        )
    except Exception as e:
        print(f"❌ Error during execution: {e}")
        import traceback
        traceback.print_exc()

# ==========================================
# 4. 実行
# ==========================================
